In [1]:
"""
Prepare the DNA dataset for training, 
"""
import os
import pickle
import requests
import numpy as np

In [2]:
from Bio import Entrez, SeqIO
import os

def download_sequences(email, search_term, max_size_mb, batch_size=100):
    Entrez.email = email
    handle = Entrez.esearch(db="nucleotide", term=search_term, retmax=batch_size)
    record = Entrez.read(handle)
    handle.close()

    ids = record["IdList"]
    print(ids)
    downloaded_size = 0
    dataset = ""
    
    for start in range(0, len(ids), batch_size):
        end = min(len(ids), start+batch_size)
        print(f"Downloading records {start+1} to {end}...")

        fetch_handle = Entrez.efetch(db="nucleotide", id=ids[start:end], rettype="fasta", retmode="text")
        data = fetch_handle.read()
        fetch_handle.close()

        filename = f"sequences.fasta"
        with open(filename, "w") as f:
            f.write(data)

        downloaded_size += len(data)
        max_size = max_size_mb * 1024 * 1024
        print(downloaded_size, max_size)

        dataset += data
        if downloaded_size >= max_size_mb * 1024 * 1024:
            break

        print(f"Downloaded so far: {downloaded_size / (1024 * 1024):.2f} MB")
    return dataset

data = open("/home/aaron/Downloads/GRCh38_latest_genomic.fna").read()

In [4]:
GB = 1024 * 1024 * 1024
data = data[:int(0.1*GB)]

In [9]:
# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

all the unique characters: 
 ,.01348>ACGHNPRT_abceghilmnoprsty
vocab size: 35


In [5]:
# create the train and test splits
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

In [12]:
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
# encode both to integers
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

train has 96,636,763 tokens
val has 10,737,419 tokens


In [6]:
# use dnabert2 tokenizer
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
train_ids = tokenizer(train_data)['input_ids']
val_ids = tokenizer(val_data)['input_ids']

In [13]:
# export to bin files

wd = current_dir = os.getcwd()
print("working directory:", wd)
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(os.path.join(wd, 'train.bin'))
val_ids.tofile(os.path.join(wd, 'val.bin'))

# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open(os.path.join(wd, 'meta.pkl'), 'wb') as f:
    pickle.dump(meta, f)

# length of dataset in characters:  1115394
# all the unique characters:
#  !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
# vocab size: 65
# train has 1003854 tokens
# val has 111540 tokens

working directory: /home/aaron/DataspellProjects/nanoGPT/data/dna_gpt
